In [1]:
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm
from common.util.data_pipeline.dataset_chainer import DatasetChainer
from common.util.enums import EnvironmentType

environment = EnvironmentType.PYTORCH
datasets = DatasetChainer(
    datasets=[
        mmcbnu(included_portion=0.001, environment_type=environment),
        fvusm(included_portion=0, environment_type=environment),
    ]
)
train, test, validation = datasets.get_dataset(environment)

2023-08-22 18:23:12,076 - root - INFO - Preprocessing MMCBNU6000 dataset for train split.
100%|██████████| 4/4 [00:00<00:00, 90.14it/s]
2023-08-22 18:23:12,130 - root - INFO - Preprocessing MMCBNU6000 dataset for test split.
100%|██████████| 1/1 [00:00<00:00, 1049.10it/s]
2023-08-22 18:23:12,133 - root - INFO - Preprocessing MMCBNU6000 dataset for validation split.
100%|██████████| 1/1 [00:00<00:00, 970.01it/s]
2023-08-22 18:23:12,145 - root - INFO - Preprocessing FV_USM dataset for train split.
0it [00:00, ?it/s]
2023-08-22 18:23:12,148 - root - INFO - Preprocessing FV_USM dataset for test split.
0it [00:00, ?it/s]
2023-08-22 18:23:12,150 - root - INFO - Preprocessing FV_USM dataset for validation split.
0it [00:00, ?it/s]
2023-08-22 18:23:12,153 - root - INFO - Concatenating train set
2023-08-22 18:23:12,155 - root - INFO - Concatenating test set
2023-08-22 18:23:12,155 - root - INFO - Concatenating validation set


In [2]:
from common.util.enums import DatasetSplitType


# datasets.get_files(DatasetSplitType.TRAIN)

In [3]:
import torch
import torch.nn as nn


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # Layer 1: CNN with filter size 3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=0)
        self.relu1 = nn.ReLU()

        # Layer 2: CNN with filter size 3 and stride 2
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=0)
        self.relu2 = nn.ReLU()

        # Layer 3: Dense layer
        self.fc = nn.Linear(in_features=72, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply layer 1
        x = self.conv1(x)

        x = self.relu1(x)

        # Apply layer 2
        x = self.conv2(x)
        x = self.relu2(x)

        # Flatten the tensor before passing through the dense layer
        x = x.view(x.size(0), -1)

        # Apply layer 3
        x = self.fc(x)
        x = self.sigmoid(x)

        return x


# Instantiate the model
model = MyModel()

# Print the model architecture
print(model)

MyModel(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(2, 2))
  (relu2): ReLU()
  (fc): Linear(in_features=72, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [4]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

    # Validation loop (optional)
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation:
            outputs = model(inputs.float())
            val_loss += criterion(outputs, labels.float())
            predicted = (outputs > 0.8).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(
        f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Accuracy: {(correct/total)*100:.2f}%"
    )
model.train()

Epoch [1/10], Loss: 0.6674, Val Loss: 0.6661, Accuracy: 0.00%
Epoch [2/10], Loss: 0.6641, Val Loss: 0.6630, Accuracy: 0.00%
Epoch [3/10], Loss: 0.6609, Val Loss: 0.6599, Accuracy: 0.00%
Epoch [4/10], Loss: 0.6577, Val Loss: 0.6565, Accuracy: 0.00%
Epoch [5/10], Loss: 0.6544, Val Loss: 0.6531, Accuracy: 0.00%
Epoch [6/10], Loss: 0.6512, Val Loss: 0.6496, Accuracy: 0.00%
Epoch [7/10], Loss: 0.6480, Val Loss: 0.6460, Accuracy: 0.00%
Epoch [8/10], Loss: 0.6448, Val Loss: 0.6424, Accuracy: 0.00%
Epoch [9/10], Loss: 0.6415, Val Loss: 0.6388, Accuracy: 0.00%
Epoch [10/10], Loss: 0.6382, Val Loss: 0.6351, Accuracy: 0.00%


MyModel(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(2, 2))
  (relu2): ReLU()
  (fc): Linear(in_features=72, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)